In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import utils_precificacao

In [3]:
# df = pd.read_json("assets/olx_location.json", orient="records", convert_dates=False)
# df.caracteristicas.apply(pd.Series)

In [4]:
df = utils_precificacao.handle_dataframe("assets/olx_location_cep.json")
df.head()

,_id,codigo,titulo,preco,data_publicacao,link,descricao,categoria,tipo,area_util,...,banheiros,vagas_na_garagem,cep,municipio,bairro,logradouro,detalhes_do_imovel,detalhes_do_condominio,condominio,iptu
0,{'$oid': '607458fa77dee513e95c7f1c'},749629484,Parque Alameda Real - Apartamento 2 quartos em...,138900.0,12/04 às 11:19,https://se.olx.com.br/sergipe/imoveis/parque-a...,Código do anúncio: 3716\nParque Alameda Real |...,Apartamentos,Venda - apartamento padrão,40.0,...,0.0,0.0,49044350,Aracaju,Santa Maria,Avenida A200 (Lot P do Sul),NaN,NaN,NaN,NaN
1,{'$oid': '607458ff77dee513e95c7f1d'},868448367,::Ecoville park,330000.0,12/04 às 10:48,https://se.olx.com.br/sergipe/imoveis/ecoville...,Ecoville park\n Localizado no bairro ponto no...,Apartamentos,Venda - apartamento padrão,NaN,...,2.0,1.0,49025100,Aracaju,Jardins,Avenida Deputado Sílvio Teixeira,NaN,NaN,NaN,NaN
2,{'$oid': '6074590377dee513e95c7f1e'},868448657,::Ecoville park,330000.0,12/04 às 10:49,https://se.olx.com.br/sergipe/imoveis/ecoville...,Ecoville park\n Localizado no bairro ponto no...,Apartamentos,Venda - apartamento padrão,NaN,...,2.0,1.0,49025100,Aracaju,Jardins,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '6074590677dee513e95c7f1f'},868449048,Bossa Nova Condomínio Clube,200001.0,12/04 às 10:50,https://se.olx.com.br/sergipe/imoveis/bossa-no...,"Oportunidade!\nApartamento com, 2/4, suíte, va...",Apartamentos,Venda - apartamento padrão,61.0,...,2.0,1.0,49095815,Aracaju,Jabotiana,Rua Nourival da Silva,"Área de serviço, Ar condicionado, Churrasqueir...","Condomínio fechado, Segurança 24h, Portaria, P...",NaN,NaN
4,{'$oid': '6074590a77dee513e95c7f20'},868449075,::Ecoville park,330000.0,12/04 às 10:50,https://se.olx.com.br/sergipe/imoveis/ecoville...,Ecoville park\n Localizado no bairro ponto no...,Apartamentos,Venda - apartamento padrão,NaN,...,2.0,1.0,49025100,Aracaju,Jardins,Avenida Deputado Sílvio Teixeira,NaN,NaN,NaN,NaN


In [16]:
from sklearn.model_selection import train_test_split

df, df_test = train_test_split(df, test_size=0.1)

In [17]:
columns = ['preco', 'bairro' ,'area_util', 'quartos', 'banheiros', 'vagas_na_garagem', 'condominio', 'iptu']

df_train = utils_precificacao.handle_attributes(df, columns)
n_variables = ['area_util', 'quartos', 'banheiros', 'vagas_na_garagem', 'condominio', 'iptu']
cat_variables = ['bairro']
target = 'preco'

In [18]:
from category_encoders import OrdinalEncoder
enc = OrdinalEncoder(cols=cat_variables)
enc.fit(df_train[cat_variables]);

In [19]:
df_train[cat_variables] = enc.transform(df_train[cat_variables])
X = df_train[n_variables + cat_variables]
y = df_train[target]

In [20]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=200, n_jobs= -1,random_state=0)
results, results_values = utils_precificacao.test_model(model, X, y)
np.mean(results)

10610620.258825881

In [21]:
# LGBM Regressor
from lightgbm import LGBMRegressor
model = LGBMRegressor(objective='regression')
results, results_values = utils_precificacao.test_model(model, X, y)
np.mean(results)

8638696.192471141

In [22]:
# Linear Regressor
from sklearn.linear_model import LinearRegression
model = LinearRegression(n_jobs=-1)
results, results_values = utils_precificacao.test_model(model, X, y)
np.mean(results)

8289159.644636436

In [23]:
model = RandomForestRegressor(n_estimators=2000, n_jobs=-1, random_state=0)
# model = LGBMRegressor(objective='regression')
# model = LinearRegression(n_jobs=-1)
model = utils_precificacao.train_model(model, df_train, n_variables + cat_variables, target)

In [24]:
data = {'area_util': [126,111,55,87,74],
        'quartos': [3,3,2,3,3],
        'banheiros': [2,3,float('Nan'),2,3],
        'vagas_na_garagem': [1,2,1,1,1], 
        'condominio': [840,550,float('Nan'),235,440],
        'iptu': [float('Nan'),1700,float('Nan'),400,float('Nan')],  
        'bairro': ['Suíssa','Jardins','Jabotiana','Ponto Novo','Luzia'],
        'preco': [219900, 600000, 120000, 160000, 360000]}

df_test = utils_precificacao.handle_attributes(df_test, columns)
df_test = df_test.fillna(0)

In [25]:
df_test[cat_variables] = enc.transform(df_test[cat_variables])

X_prev = df_test[n_variables + cat_variables]
df_test['previsão R$'] = np.round(model.predict(X_prev))
df_test['erro absoluto'] = df_test['preco'] -  df_test['previsão R$']
df_test['erro relativo'] = np.round((df_test['preco'] -  df_test['previsão R$'])/df_test['preco'], 2)
df_test

,preco,bairro,area_util,quartos,banheiros,vagas_na_garagem,condominio,iptu,previsão R$,erro absoluto,erro relativo
429,155000.0,17.0,70.0,3.0,2.0,1.0,359.0,174.0,248546.0,-93546.0,-0.60
2789,260000.0,10.0,0.0,2.0,3.0,1.0,630.0,1244.0,275562.0,-15562.0,-0.06
422,1100000.0,4.0,168.0,3.0,2.0,2.0,1150.0,174.0,584043.0,515957.0,0.47
486,1350577.0,3.0,219.0,3.0,4.0,4.0,359.0,174.0,1323212.0,27365.0,0.02
4565,125000.0,1.0,65.0,3.0,1.0,1.0,240.0,174.0,161875.0,-36875.0,-0.30
...,...,...,...,...,...,...,...,...,...,...,...
3578,296000.0,4.0,73.5,3.0,2.0,1.0,359.0,174.0,335506.0,-39506.0,-0.13
1476,199000.0,7.0,73.5,3.0,2.0,1.0,345.0,174.0,261790.0,-62790.0,-0.32
3567,296000.0,4.0,73.5,3.0,2.0,1.0,359.0,174.0,335506.0,-39506.0,-0.13
338,160701.0,1.0,60.0,2.0,2.0,1.0,359.0,174.0,943774.0,-783073.0,-4.87


In [26]:
df_test['erro relativo'].abs().mean()

1.781943231441048

In [31]:
import joblib

joblib.dump(model, 'numpy_models/1_model_random_forest_simple.pkl')

['numpy_models/1_model_random_forest_simple.pkl']

In [33]:
joblib.dump(enc, 'numpy_models/enc_for_1_districts.pkl')

['numpy_models/enc_for_1_districts.pkl']